# 应用3 | Code Agent：让 AI 编写和执行代码

---

## 🎯 本节目标

构建一个**真正能编写和执行代码的 AI Agent**：
- 使用真实 LLM 生成 Python 代码
- 安全的沙箱执行环境
- 错误自动修复机制

---

## Code Agent 的能力

1. **理解问题** → 将自然语言转换为代码需求
2. **生成代码** → LLM 编写 Python 代码
3. **安全执行** → 在沙箱中运行代码
4. **自我修复** → 根据错误信息修复代码

## 运行前：准备 Ollama 后端

如果你选择 Ollama 作为后端，请先完成准备步骤：
- 参见 [PREPARE_OLLAMA.ipynb](./PREPARE_OLLAMA.ipynb)

完成后再继续本 notebook。


In [ ]:
import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

import io
import re
import traceback
from typing import Dict, Any, List, Optional, Tuple
from contextlib import redirect_stdout, redirect_stderr
from dataclasses import dataclass

from utils.llm_backend import get_llm_backend

print("✓ 环境准备完成!")

In [ ]:
def check_ollama_ready(host="http://localhost:11434"):
    try:
        import requests
        resp = requests.get(f"{host}/api/tags", timeout=2)
        return resp.status_code == 200
    except Exception:
        return False

# 配置 LLM
if not check_ollama_ready():
    print("Ollama 未启动。请先执行: ollama serve")
    print("并下载模型: ollama pull qwen3:4b")
try:
    llm = get_llm_backend("ollama", model="qwen3:4b")
    test = llm.chat([{"role": "user", "content": "print('test')"}])
    print(f"✓ LLM 连接成功")
except Exception as e:
    print(f"⚠️ Ollama 不可用: {e}")
    try:
        llm = get_llm_backend("openai", model="gpt-3.5-turbo")
        print("✓ 使用 OpenAI 后端")
    except:
        llm = None
        print("❌ 请配置 LLM 后端")

---

## 1. 安全代码执行器

执行任意代码是**危险的**！我们需要：
- 限制可用的内置函数
- 限制可导入的模块
- 捕获输出和错误

In [ ]:
class SafeCodeExecutor:
    """
    安全的 Python 代码执行器
    
    安全特性:
    - 白名单内置函数
    - 限制可导入模块
    - 禁止文件/网络操作
    - 超时保护 (生产环境需要)
    """
    
    # 安全的内置函数
    SAFE_BUILTINS = {
        'abs', 'all', 'any', 'bool', 'dict', 'divmod', 'enumerate',
        'filter', 'float', 'format', 'frozenset', 'hash', 'int',
        'isinstance', 'issubclass', 'iter', 'len', 'list', 'map',
        'max', 'min', 'next', 'pow', 'print', 'range', 'repr',
        'reversed', 'round', 'set', 'slice', 'sorted', 'str',
        'sum', 'tuple', 'type', 'zip', 'True', 'False', 'None', 'Exception', 'ValueError', 'RuntimeError'
    }
    
    # 允许导入的模块
    ALLOWED_MODULES = {
        'math', 'random', 'statistics', 'collections', 'itertools',
        'functools', 'operator', 'string', 'datetime', 'json', 're'
    }
    
    # 危险模式检测
    DANGEROUS_PATTERNS = [
        (r'\bopen\s*\(', 'File operations not allowed'),
        (r'\bexec\s*\(', 'exec() not allowed'),
        (r'\beval\s*\(', 'eval() not allowed'),
        (r'\b__import__\s*\(', '__import__() not allowed'),
        (r'\bos\.', 'os module not allowed'),
        (r'\bsys\.', 'sys module access not allowed'),
        (r'\bsubprocess', 'subprocess not allowed'),
        (r'\bsocket', 'network operations not allowed'),
        (r'\brequests', 'HTTP requests not allowed'),
        (r'\bshutil', 'shutil not allowed'),
        (r'\bpickle', 'pickle not allowed'),
    ]
    
    def __init__(self):
        # 构建安全的 builtins
        import builtins
        self.safe_builtins = {
            name: getattr(builtins, name)
            for name in self.SAFE_BUILTINS
            if hasattr(builtins, name)
        }
        self.safe_builtins['__import__'] = self._safe_import
    
    def _safe_import(self, name, *args, **kwargs):
        """安全的 import 函数"""
        if name not in self.ALLOWED_MODULES:
            raise ImportError(f"Module '{name}' is not allowed. Allowed: {self.ALLOWED_MODULES}")
        return __import__(name, *args, **kwargs)
    
    def check_safety(self, code: str) -> Tuple[bool, str]:
        """检查代码安全性"""
        for pattern, message in self.DANGEROUS_PATTERNS:
            if re.search(pattern, code):
                return False, f"Security Error: {message}"
        return True, "Code appears safe"
    
    def execute(self, code: str, timeout: int = 10) -> Dict[str, Any]:
        """
        执行代码
        
        Returns:
            dict: {success, output, error, variables}
        """
        # 安全检查
        is_safe, reason = self.check_safety(code)
        if not is_safe:
            return {
                'success': False,
                'output': '',
                'error': reason,
                'variables': {}
            }
        
        # 捕获输出
        stdout_capture = io.StringIO()
        stderr_capture = io.StringIO()
        
        # 执行命名空间
        namespace = {'__builtins__': self.safe_builtins}
        
        try:
            with redirect_stdout(stdout_capture), redirect_stderr(stderr_capture):
                exec(code, namespace)
            
            # 提取非内置变量
            variables = {
                k: repr(v)[:200] for k, v in namespace.items()
                if not k.startswith('_') and k != '__builtins__'
            }
            
            return {
                'success': True,
                'output': stdout_capture.getvalue(),
                'error': stderr_capture.getvalue(),
                'variables': variables
            }
        
        except Exception as e:
            return {
                'success': False,
                'output': stdout_capture.getvalue(),
                'error': f"{type(e).__name__}: {str(e)}",
                'variables': {}
            }

# 创建执行器
executor = SafeCodeExecutor()
print("✓ 安全执行器已创建")


In [ ]:
# 测试安全执行器
print("测试 1: 正常代码")
result = executor.execute("""
import math
x = math.sqrt(16)
print(f"sqrt(16) = {x}")
""")
print(f"Success: {result['success']}")
print(f"Output: {result['output']}")

print("\n测试 2: 危险代码 (文件操作)")
result = executor.execute("open('test.txt', 'w')")
print(f"Success: {result['success']}")
print(f"Error: {result['error']}")

print("\n测试 3: 禁止的模块")
result = executor.execute("import os; os.system('ls')")
print(f"Success: {result['success']}")
print(f"Error: {result['error']}")

---

## 2. Code Agent 实现

In [ ]:
class CodeAgent:
    """
    Code Agent - 使用真实 LLM 生成和执行代码
    """
    
    SYSTEM_PROMPT = """You are a Python code generator. Your task is to write Python code to solve problems.

Rules:
1. Return ONLY executable Python code, no explanations
2. Use print() to show results
3. Available modules: math, random, statistics, collections, itertools, functools, datetime, json, re
4. NO file operations (open, read, write)
5. NO network operations
6. NO os or sys modules
7. Keep code simple and focused
8. Must output at least one executable statement

Format your response as:
```python
# your code here
```
"""
    
    FIX_PROMPT = """The previous code had an error. Please fix it.

Previous code:
```python
{code}
```

Error message:
{error}

Please provide the fixed code:
"""
    
    def __init__(self, llm, executor: SafeCodeExecutor = None, max_retries: int = 3):
        self.llm = llm
        self.executor = executor or SafeCodeExecutor()
        self.max_retries = max_retries
        self.history: List[Dict] = []
    
    def _extract_code(self, response: str) -> str:
        """从 LLM 响应中提取代码"""
        # 尝试提取 ```python ... ``` 块
        pattern = r'```python\s*\n(.*?)\n```'
        match = re.search(pattern, response, re.DOTALL)
        if match:
            return match.group(1).strip()
        
        # 尝试提取 ``` ... ``` 块
        pattern = r'```\s*\n(.*?)\n```'
        match = re.search(pattern, response, re.DOTALL)
        if match:
            return match.group(1).strip()
        
        # 如果没有代码块，假设整个响应是代码
        return response.strip()

    def _fallback_code(self, problem: str) -> str:
        """当 LLM 输出为空时，使用规则兜底"""
        text = problem.lower()

        if ("prime" in text and "50" in text) or ("素数" in problem and "50" in problem):
            return """def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = [n for n in range(1, 51) if is_prime(n)]
print(primes)
"""

        if ("bubble sort" in text) or ("冒泡" in problem):
            return """arr = [64, 34, 25, 12, 22, 11, 90]

n = len(arr)
for i in range(n):
    for j in range(0, n - i - 1):
        if arr[j] > arr[j + 1]:
            arr[j], arr[j + 1] = arr[j + 1], arr[j]

print(arr)
"""

        if ("palindrome" in text) or ("回文" in problem):
            return """import re

def is_palindrome(s):
    cleaned = re.sub(r"[^a-z0-9]", "", s.lower())
    return cleaned == cleaned[::-1]

tests = ["racecar", "hello", "A man a plan a canal Panama"]
for s in tests:
    print(f"{s} -> {is_palindrome(s)}")
"""

        if "fibonacci" in text or "斐波那契" in problem:
            return """nums = [0, 1]
while len(nums) < 20:
    nums.append(nums[-1] + nums[-2])

print(nums)
"""

        if ("mean" in text and "median" in text and "standard" in text) or (
            "均值" in problem and ("中位数" in problem or "中值" in problem)
        ):
            return """import statistics

nums = [23, 45, 67, 89, 12, 34, 56, 78, 90, 11]
print(f"Mean: {statistics.mean(nums)}")
print(f"Median: {statistics.median(nums)}")
print(f"Standard deviation: {statistics.pstdev(nums)}")
print(f"Min: {min(nums)}")
print(f"Max: {max(nums)}")
"""

        return ""
    
    def generate_code(self, problem: str) -> str:
        """生成代码"""
        messages = [
            {"role": "system", "content": self.SYSTEM_PROMPT},
            {"role": "user", "content": problem}
        ]
        response = self.llm.chat(messages, temperature=0.2)
        code = self._extract_code(response)

        if not code.strip():
            fallback = self._fallback_code(problem)
            if fallback:
                return fallback

        return code
    
    def fix_code(self, code: str, error: str) -> str:
        """修复代码"""
        messages = [
            {"role": "system", "content": self.SYSTEM_PROMPT},
            {"role": "user", "content": self.FIX_PROMPT.format(code=code, error=error)}
        ]
        response = self.llm.chat(messages, temperature=0.2)
        fixed = self._extract_code(response)

        if not fixed.strip():
            return code

        return fixed
    
    def solve(self, problem: str, verbose: bool = True) -> Dict[str, Any]:
        """
        解决问题
        
        Args:
            problem: 问题描述
            verbose: 是否显示详细过程
            
        Returns:
            dict: {success, output, code, attempts}
        """
        if verbose:
            print(f"\n{'='*60}")
            print(f"Problem: {problem}")
            print(f"{'='*60}")
        
        # 生成初始代码
        if verbose:
            print("\nGenerating code...")
        
        code = self.generate_code(problem)
        if not code.strip():
            code = "raise ValueError('Empty code')"

        attempts = []
        
        for attempt in range(self.max_retries):
            if verbose:
                print(f"\n--- Attempt {attempt + 1}/{self.max_retries} ---")
                print(f"Code:\n```python\n{code}\n```")
            
            # 执行代码
            result = self.executor.execute(code)
            
            attempts.append({
                'code': code,
                'result': result
            })
            
            if result['success']:
                if verbose:
                    print("\nExecution successful!")
                    print(f"Output:\n{result['output']}")
                
                # 记录历史
                self.history.append({
                    'problem': problem,
                    'code': code,
                    'output': result['output'],
                    'success': True,
                    'attempts': len(attempts)
                })
                
                return {
                    'success': True,
                    'output': result['output'],
                    'code': code,
                    'attempts': attempts
                }
            
            if verbose:
                print(f"\nError: {result['error']}")
            
            # 尝试修复
            if attempt < self.max_retries - 1:
                if verbose:
                    print("\nAttempting to fix...")
                code = self.fix_code(code, result['error'])
        
        # 所有尝试都失败
        self.history.append({
            'problem': problem,
            'code': code,
            'error': result['error'],
            'success': False,
            'attempts': len(attempts)
        })
        
        return {
            'success': False,
            'output': '',
            'error': result['error'],
            'code': code,
            'attempts': attempts
        }

# 创建 Code Agent
if llm:
    agent = CodeAgent(llm, executor)
    print("\nCode Agent 已创建!")
else:
    print("需要配置 LLM 后端")


---

## 3. 测试 Code Agent

In [ ]:
# 测试 1: 数学计算
result = agent.solve("Calculate the factorial of 10 and print it")

In [ ]:
# 测试 2: 列表操作
result = agent.solve("Create a list of the first 20 fibonacci numbers and print them")

In [ ]:
# 测试 3: 素数
result = agent.solve("Find all prime numbers between 1 and 50, then print them")

In [ ]:
# 测试 4: 统计计算
result = agent.solve("""
Given the numbers [23, 45, 67, 89, 12, 34, 56, 78, 90, 11],
calculate and print:
- Mean
- Median
- Standard deviation
- Min and Max
""")

In [ ]:
# 测试 5: 字符串处理
result = agent.solve("""
Write a function to check if a string is a palindrome.
Test it with these strings: 'racecar', 'hello', 'A man a plan a canal Panama'
Print the results for each.
""")

In [ ]:
# 测试 6: 排序算法
result = agent.solve("""
Implement bubble sort algorithm and use it to sort [64, 34, 25, 12, 22, 11, 90].
Print the sorted result.
""")

---

## 本节总结


1. **安全执行环境**
   - 白名单限制内置函数和模块
   - 危险模式检测
   - 输出捕获

2. **代码生成**
   - 使用 LLM 将自然语言转换为代码
   - 代码提取与解析

3. **自我修复**
   - 错误分析
   - 迭代修复

### 扩展练习

1. **添加超时保护**: 使用 multiprocessing 限制执行时间
2. **支持更多语言**: 扩展到 JavaScript、Shell
3. **可视化执行**: 展示代码执行过程